<a href="https://colab.research.google.com/github/jonesavelino/idea-tool/blob/main/IDEA_ConverterDoccanoSpacy3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETAPA 1 - CONVERTER O TEXTOS ANOTADOS PELO DOCCANO

## Caderno resposável por receber o arquivo exportado do Doccano, no padrão JSON Lines, com as entidades (entities) e relações (relations) destacadas por posição no arquivo.

### O objetivo é transformar esse arquivo em JSON para SPACY

### Pré-requisitos:

#### 1- Arquivo gerado pelo Doccano
#### 2- Montagem do Drive

### Parâmetros:

#### DATA_FILE_PATH_IN = Arquivo gerado pelo Doccano (/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/inputs/exporta_anotacao_DD_MM_YYYY.jsonl)

#### DATA_FILE_PATH_OUT = Arquivo de Saída no padrão do SPACY (/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao_17_05_2023.spacy)
--
# Data: 18-05-2023
# Autores: Maj Giselle Farias e Jones.

Fonte: https://www.kaggle.com/code/abneetwats24/doccano-to-spacy3-data-conversion-and-training/notebook

### 1 - Montagem do Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2 - Imports e bibliotecas

In [ ]:
!pip uninstall spacy -y

Found existing installation: spacy 3.5.2
Uninstalling spacy-3.5.2:
  Successfully uninstalled spacy-3.5.2


In [ ]:
!pip install -U spacy==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (659 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl (126 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 10.3 MB/s eta 0:00:00
  Created wheel for spacy: filename=spacy-3.1.0-cp310-cp310-linux_x86_64.whl size=29104722 sha256=2bfe16ca2efcc17de5ee5081a9bba20c825a62e29d419a89e58bf1b14d518760
  Stored in directory: /root/.cache/pip/wheels/2a/cf/84/6c88a39f9a3c8c8b3e928077b96f2dec87ac896254e1770817
Successfully built spacy
  Attempting uninstall: wasabi
    Found existing installation

In [ ]:
!pip install -U allennlp
!python -m spacy download pt_core_news_sm

In [ ]:
#!python -m spacy download pt_core_news_lg

In [ ]:
# Trecho do código "Auxiliares"

!pip install jsonlines

import os
import gzip
import json
import random
import math
import jsonlines


def pega_todos_ids():
    # folder path
    dir_path = r'/Users/gisellerosa/Downloads/sciERC_raw_data'

    # list to store files
    res = []
    res_ann = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            res.append(path)
        if(path.endswith('.ann')):
            res_ann.append(path[:-4])
    print(res_ann)

    return res_ann


def dicts_to_jsonl(data: list, filename: str, compress: bool = True) -> None:
    """
    Method saves list of dicts into jsonl file.

    :param data: (list) list of dicts to be stored,
    :param filename: (str) path to the output file. If suffix .jsonl is not given then methods appends
        .jsonl suffix into the file.
    :param compress: (bool) should file be compressed into a gzip archive?
    """

    sjsonl = '.jsonl'
    sgz = '.gz'

    # Check filename

    if not filename.endswith(sjsonl):
        filename = filename + sjsonl

    # Save data

    if compress:
        filename = filename + sgz
        with gzip.open(filename, 'w') as compressed:
            for ddict in data:
                jout = json.dumps(ddict) + '\n'
                jout = jout.encode('utf-8')
                compressed.write(jout)
    else:
        with open(filename, 'w') as out:
            for ddict in data:
                jout = json.dumps(ddict) + '\n'
                out.write(jout)


def valida_start(texto, start):
    if texto[start] != ' ':
        pos = start - 1
        while texto[pos] != ' ':
            pos -= 1
    else:
        pos = start
        while texto[pos] == ' ':
            pos += 1
    return pos


def valida_end(texto, end):
    if texto[end] == ' ':
        pos = end - 1
        while pos == ' ':
            pos -= 1
        return pos
    else:
        pos = end - 1
        if (pos + 1) >= len(texto):
            pos = end
        elif texto[end + 1] == ' ':
                pos = end
        while texto[pos + 1] != ' ':
            pos -= 1
        return pos



def valida_end2(texto, end):
    texto = texto + '  '
    if not texto[end].isalnum():
        if texto[end-1].isalnum():
            return end-1
        else:
            end -= 1
            while not texto[end].isalnum():
                end -= 1
            return end
    else:
        if texto[end-1].isalnum():
            pos = end
            while texto[pos + 1].isalnum():
                pos += 1
            return pos - 1
        else:
            end -= 1
            while texto[end].isalnum():
                end -= 1
                if texto[end - 1].isalnum():
                    return end - 1
                else:
                    end -= 1
                    while not texto[end].isalnum():
                        end -= 1
                    return end
            return end


def train_test_split_total(data_file_path, percent_split, split1_path, split2_path):
    with open(data_file_path, 'r') as jsonl_file:
        json_total_list = []
        num_lines = 0

        for line in jsonl_file:
            json_total_list.append(json.loads(line))
            num_lines += 1

        print('Quantidade de linhas:', num_lines)

        qtd_split = math.floor(percent_split * num_lines)

        print('qtd_split_train:', qtd_split)

        randomlist = random.sample(range(0, num_lines), qtd_split)

        json_1_list = []
        json_2_list = []

        for line_num in range(num_lines):
            if line_num in randomlist:
                json_1_list.append(json_total_list[line_num])
            else:
                json_2_list.append(json_total_list[line_num])

        print('tamanho json_train_list:', len(json_1_list))
        print('tamanho json_dev_list:', len(json_2_list))

    with open(split1_path, 'w') as f_saida:
        for js in json_1_list:
            jout = json.dumps(js) + '\n'
            f_saida.write(jout)

    with open(split2_path, 'w') as f_saida_2:
        for js in json_2_list:
            jout = json.dumps(js) + '\n'
            f_saida_2.write(jout)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import os
import random
import logging
# from sklearn.metrics import classification_report
# from sklearn.metrics import precision_recall_fscore_support
# # from spacy.gold import GoldParse
# # from spacy.scorer import Scorer
# from sklearn.metrics import accuracy_score
import spacy

print(spacy.__version__)
import re
import random

# spacy.prefer_gpu()

# DATA_FILE_PATH_IN = "inputs/scierc-test.jsonl"
# DATA_FILE_PATH_OUT = "outputs/scierc-test.spacy"

DATA_FILE_PATH_IN = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/inputs/exporta_anotacao.jsonl"
DATA_FILE_PATH_OUT = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao.spacy"

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for id_ent, label, start, end in entities:
            # print('analisando id: ', id)
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label, id_ent])
        cleaned_data.append([text, {'entities': valid_entities, 'relations': annotations['relations']}])
    return cleaned_data


def validate_overlap(ALL_DATA):
    for ix, x in enumerate(ALL_DATA):
        startCK = []
        for iy, y in enumerate(x[-1]['entities']):
            if iy == 0:
                startCK.append([y[0], y[1]])
            else:
                pop = False
                for z in startCK:
                    if z[0] <= y[0] < z[1]:
                        print(y, z)
                        ALL_DATA[ix][-1]['entities'].pop(iy)
                        print(ALL_DATA[ix][-1]['entities'].pop(iy))
                        pop = True
                        break
                if pop == False:
                    startCK.append([y[0], y[1]])
    return ALL_DATA


def convert_doccano_to_spacy(doccano_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(doccano_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['text']
            entities = data['entities']
            relations = data['relations']
            if len(entities) > 0:
                for i, entity in enumerate(entities):
                    entities[i] = list(entity.values())
                for j, relation in enumerate(relations):
                    relations[j] = list(relation.values())
                training_data.append((text, {"entities": entities, "relations": relations}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None


def conversao_jsonl_spacy(data_file_path_in, data_file_path_out):
    ALL_DATA = convert_doccano_to_spacy(data_file_path_in)

    ALL_DATA = trim_entity_spans(ALL_DATA)
    ALL_DATA = validate_overlap(ALL_DATA)
    random.shuffle(ALL_DATA)

    # print("RESULTADO FINAL")
    # print(ALL_DATA)

    import pandas as pd
    from tqdm import tqdm
    import spacy
    from spacy.tokens import DocBin, Doc
    from wasabi import Printer

    # MAP_LABELS = {
    #     "USADA_PARA": "usada_para"
    # }

    MAP_LABELS = {
        "tratado_em": "tratado_em",
        "composto_de": "composto_de",
        "instancia_de": "instancia_de",
        "finalidade_de": "finalidade_de",
        "aplicado_em": "aplicado_em",
        "ocorre_em": "ocorre_em",
        "correferencia_de": "correferencia_de",
        "responsavel_por": "responsavel_por",
        "complementado_por": "complementado_por",
        "topico_de": "topico_de",
        "capacidade_de": "capacidade_de",
        "tipo_de": "tipo_de",
        "definido_por": "definido_por",
        "referenciado_a": "referenciado_a",
        "capacidade_de": "capacidade_de",
        "ocorre_em": "ocorre_em"
    }

    nlp = spacy.blank("pt")  # load a new pipeline spacy model com o vocabulário em português
    Doc.set_extension("rel", default={}, force=True)
    db = DocBin(store_user_data=True)  # create a DocBin object
    # db = DocBin(docs=docs["total"], store_user_data=True) ???????????????

    msg = Printer()

    c = 0
    for text, annot in tqdm(ALL_DATA):  # data in previous format
        doc = nlp.make_doc(text)  # create doc object from text
        ents = []
        span_starts = {}
        neg = 0
        pos = 0
        print('\n\n-----------------')
        print('-----------------\nTEXT:', text)
        for start, end, label, id_ent in annot["entities"]:  # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
            if span is None:
                #             print("======================================================Skipping entity Start===================================================")
                #             print(start, end, label, span)
                #             print(doc.text[0:end-1],doc.text[start],doc.text[end],'kh',sep='|')
                #             print("======================================================Skipping entity End===================================================")
                print('------------------------Erro de span------------------------')
                print('\t', 'id_ent:', id_ent)
                print('------------------------------------------------------------')
                c += 1
                s = doc.text
                sub_E = s[end:]
                sub_S = s[:start]
                end = end + (0 if len(sub_E.split(" ", 1)[0]) <= 0 else len(sub_E.split(" ", 1)[0]))
                start = start - (0 if len(sub_S.rsplit(" ", 1)[-1]) <= 0 else len(sub_S.rsplit(" ", 1)[-1]))
                #             print(s[start:end])
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
                if span is None:
                    print("++++++++++++++++++++++++++++Skipping entity Start++++++++++++++++++++++++++++")
                    print(start, end, label, span)
                    print(doc.text[start:end], doc.text[start], doc.text[end - 1], 'kh', sep='|')
                    print("++++++++++++++++++++++++++++Skipping entity End++++++++++++++++++++++++++++++")
                    c += 1
            else:
                ents.append(span)
                span_starts[
                    id_ent] = span.start  # Criando dict com informacao de id da entidade e token inicial da entidade

        doc.ents = ents  # label the text with the ents
        # spacy.displacy.render(doc, style="ent")

        # CONVERSAO DAS RELACOES

        # Criando lista com  todas as entidades
        all_starts = []
        rels = {}
        for e in doc.ents:
            all_starts.append(e.start)

        # print(all_starts)

        for x1 in all_starts:
            for x2 in all_starts:
                rels[(x1, x2)] = {}

        relations = annot["relations"]
        # [{"id":10,"from_id":52,"to_id":51,"type":"usada_para"}]
        print("len(relations):", len(relations))

        for relation in relations:
            from_id = relation[1]  # "from_id"
            start = span_starts[from_id]
            to_id = relation[2]  # "to_id"
            end = span_starts[to_id]
            label = relation[3]  # "type"
            # print('\tstart/end: ', start, end)
            # print("rels[(start, end)]:", [(start, end)])
            # label = MAP_LABELS[label]
            if label not in rels[(start, end)]:
                rels[(start, end)][label] = 1.0
                pos += 1

        # The annotation is complete, so fill in zero's where the data is missing
        for x1 in all_starts:
            for x2 in all_starts:
                for label in MAP_LABELS.values():
                    if label not in rels[(x1, x2)]:
                        neg += 1
                        rels[(x1, x2)][label] = 0.0
                        #print(rels[(x1, x2)])

        doc._.rel = rels

        print('\n')
        print('doc._.rel:')
        print(doc._.rel)

        db.add(doc)

    # #print(len(docs["total"]))
    #
    # # msg.info(
    # #     f"{len(docs['total'])} training sentences"
    # # )

    db.to_disk(data_file_path_out)  # save the docbin object
    print('erros de spans:', c)

3.3.2


In [ ]:
#from auxiliares import train_test_split_total
#from conversao_json_spacy_completo import conversao_jsonl_spacy

# Nesse trecho de código foi inserida a Validação Cruzada para Entidades

TOTAL_JSONL_PATH = DATA_FILE_PATH_IN
PERCENT_SPLIT_TEST = 0.8
PERCENT_SPLIT_DEV = 0.2

#Variável que permite dividir o conjunto de dados por X pedaços.
# Isto é, se tiver um arquivo com 1000 registros,
# e a varíavel estiver "setada" para 4, serão gerados 4 arquivos com 250 registros em cada.
NR_FOLDS = 1

# Saõ divididos o conjunto de dados em 3 partes (teste, treino e dev)
TEST_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-test"
TRAIN_PATH =  "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-train-k"
DEV_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-dev-k"

test_jsonl_path = TEST_PATH + ".jsonl"
test_spacy_path = TEST_PATH + ".spacy"

# Cria um arquivo temporário que não é utilizado
temp_path = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-temp.jsonl"

# Separando conjunto de teste dos demais (temp)
train_test_split_total(TOTAL_JSONL_PATH, PERCENT_SPLIT_TEST, temp_path, test_jsonl_path)

# Convertendo jsonl para spacy  (conjunto de teste)
conversao_jsonl_spacy(test_jsonl_path, test_spacy_path)

print(test_jsonl_path)
print(test_spacy_path)

# Criando pares de conjuntos train+dev para cada fold
for fold in range(NR_FOLDS):
    print(fold)

    train_jsonl_path_k = TRAIN_PATH + str(fold) + ".jsonl"
    train_spacy_path_k = TRAIN_PATH + str(fold) + ".spacy"
    dev_jsonl_path_k = DEV_PATH + str(fold) + ".jsonl"
    dev_spacy_path_k = DEV_PATH + str(fold) + ".spacy"

    # Separando conjuntos train e dev
    train_test_split_total(temp_path, PERCENT_SPLIT_DEV, train_jsonl_path_k, dev_jsonl_path_k)

    # Convertendo jsonl para spacy  (conjunto train)
    conversao_jsonl_spacy(train_jsonl_path_k, train_spacy_path_k)

    # Convertendo jsonl para spacy  (conjunto dev)
    conversao_jsonl_spacy(dev_jsonl_path_k, dev_spacy_path_k)

Quantidade de linhas: 15
qtd_split_train: 12
tamanho json_train_list: 12
tamanho json_dev_list: 3


100%|██████████| 3/3 [00:00<00:00, 82.60it/s]



-----------------
-----------------
TEXT: EB20-MF-10.102

DOUTRINA MILITAR TERRESTRE

ELEMENTOS DO PODER DE COMBATE - Os elementos do poder de combate terrestre representam a essência das capacidades que a F Ter emprega em situações – sejam de guerra ou de não guerra. São eles: Liderança, Informações e as Funções de Combate – Comando e Controle, Movimento e Manobra, Inteligência, Fogos, Logística e Proteção.
len(relations): 13


doc._.rel:
{(0, 0): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tipo_de': 0.0, 'definido_por': 0.0, 'referenciado_a': 0.0}, (0, 2): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tip


  0%|          | 0/2 [00:00<?, ?it/s]



-----------------
-----------------
TEXT: EB20-MF-10.102

DOUTRINA MILITAR TERRESTRE

ELEMENTOS DO PODER DE COMBATE - Os elementos do poder de combate terrestre representam a essência das capacidades que a F Ter emprega em situações – sejam de guerra ou de não guerra. São eles: Liderança, Informações e as Funções de Combate – Comando e Controle, Movimento e Manobra, Inteligência, Fogos, Logística e Proteção.
len(relations): 13


doc._.rel:
{(0, 0): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tipo_de': 0.0, 'definido_por': 0.0, 'referenciado_a': 0.0}, (0, 2): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tip

...



-----------------
-----------------
TEXT: EB20-MF-10.102

DOUTRINA MILITAR TERRESTRE

ELEMENTOS DO PODER DE COMBATE - Os elementos do poder de combate terrestre representam a essência das capacidades que a F Ter emprega em situações – sejam de guerra ou de não guerra. São eles: Liderança, Informações e as Funções de Combate – Comando e Controle, Movimento e Manobra, Inteligência, Fogos, Logística e Proteção.
len(relations): 13


doc._.rel:
{(0, 0): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tipo_de': 0.0, 'definido_por': 0.0, 'referenciado_a': 0.0}, (0, 2): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tip

  0%|          | 0/10 [00:00<?, ?it/s]



-----------------
-----------------
TEXT: EB20-MF-10.102

DOUTRINA MILITAR TERRESTRE

ELEMENTOS DO PODER DE COMBATE - Os elementos do poder de combate terrestre representam a essência das capacidades que a F Ter emprega em situações – sejam de guerra ou de não guerra. São eles: Liderança, Informações e as Funções de Combate – Comando e Controle, Movimento e Manobra, Inteligência, Fogos, Logística e Proteção.
len(relations): 13


doc._.rel:
{(0, 0): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tipo_de': 0.0, 'definido_por': 0.0, 'referenciado_a': 0.0}, (0, 2): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tip

100%|██████████| 10/10 [00:00<00:00, 63.65it/s]


-----------------
TEXT: EB20-MF-10.102

DOUTRINA MILITAR TERRESTRE

ELEMENTOS DO PODER DE COMBATE - Os elementos do poder de combate terrestre representam a essência das capacidades que a F Ter emprega em situações – sejam de guerra ou de não guerra. São eles: Liderança, Informações e as Funções de Combate – Comando e Controle, Movimento e Manobra, Inteligência, Fogos, Logística e Proteção.
len(relations): 13


doc._.rel:
{(0, 0): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tipo_de': 0.0, 'definido_por': 0.0, 'referenciado_a': 0.0}, (0, 2): {'tratado_em': 0.0, 'composto_de': 0.0, 'instancia_de': 0.0, 'finalidade_de': 0.0, 'aplicado_em': 0.0, 'ocorre_em': 0.0, 'correferencia_de': 0.0, 'responsavel_por': 0.0, 'complementado_por': 0.0, 'topico_de': 0.0, 'capacidade_de': 0.0, 'tipo_de': 0.0, 'defini